# BUG: `DatetimeIndex` slicing: `df.loc["2020-01-01": "2020-01-01"]` gives different result than `df.loc[Timestamp("2020-01-01"): Timestamp("2020-01-01")]` 

One would expect that both times the same data is selected. It is debatable which one of the two results is the "correct" one; essentially this is the question of whether one wants an inclusive range or not by default.

In any case I would definitely call it a bug though that both calls return different slices.  It is just way too easy to trip over this behavior and accidentally get a wrong slice!

The reason for the different slices seems to be that using the string or `Timestamp` return different things:

- `df.index.get_loc(a) →slice(0, 96, None) `
- `df.index.get_loc(ta) →0 `
- `df.index.get_loc(b) →slice(192, 288, None) `
- `df.index.get_loc(tb) →192 `

In particular, `df.loc["2020-01-01"]` returns all data collected on January first, whereas `df.loc[Timestamp("2020-01-01")]` returns only the single datapoint collected at `2020-01-01 00:00:00`.

The [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#indexing) says that

> This type of slicing will work on a DataFrame with a DatetimeIndex as well. Since the partial string selection is a form of label slicing, the endpoints will be included. This would include matching times on an included date

In [ ]:
from pandas import date_range, Timestamp, DataFrame, Period

ts = date_range(start="2020-01-01", end="2020-02-01", freq="15min")
df = DataFrame(range(len(ts)), index=ts)

a = "2020-01-01"
b = "2020-01-03"
ta = Timestamp(a)
tb = Timestamp(b)

print(df.index.get_loc(a), df.index.get_loc(ta))
print(df.index.get_loc(b), df.index.get_loc(tb))
print(df.loc[a:b])
print(df.loc[ta:tb])

In [ ]:
str(Timestamp(a))

In [ ]:
print(df[a:b])
print(df[ta:tb])

In [ ]:
Period("2020-01-01 to 2020-01-03")

In [ ]:
df.loc[Timestamp("2020-01-01")]

In [ ]:
df.index

In [ ]:
import pandas

pandas.show_versions()